<a href="https://colab.research.google.com/github/AirbornBird88/fb-sentiment/blob/main/SmartRections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE

In [27]:
from google.colab import drive
cwd = '/content/drive'
drive.mount(cwd)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
%cd drive/MyDrive/sentiment_data/
%ls

[Errno 2] No such file or directory: 'drive/MyDrive/sentiment_data/'
/content/drive/MyDrive/sentiment_data
catboost_info/  fb_sentiment.csv


In [118]:
# Load the CSV data
data = pd.read_csv("fb_sentiment.csv")

In [119]:
data

,Unnamed: 0,FBPost,Label
0,0,Drug Runners and a U.S. Senator have somethin...,O
1,1,"Heres a single, to add, to Kindle. Just read t...",O
2,2,If you tire of Non-Fiction.. Check out http://...,O
3,3,Ghost of Round Island is supposedly nonfiction.,O
4,4,Why is Barnes and Nobles version of the Kindle...,N
...,...,...,...
995,995,I liked it. Its youth oriented and I think th...,P
996,996,"I think the point of the commercial is that, e...",P
997,997,Kindle 3 is such a great product. I could not ...,P
998,998,develop a way to share books! that is a big d...,N


In [120]:
# Separate features and target variable
X = data['FBPost']
y = data['Label']

In [121]:
# Print class distribution in all data
print("Class distribution:")
print(y.value_counts())

Class distribution:
P    641
O    280
N     79
Name: Label, dtype: int64


In [122]:
# Compute and print class distribution in all data
class_distribution = data.groupby('Label')['Label'].count() / len(data)
print('Class distribution: ')
print(class_distribution)

Class distribution: 
Label
N    0.079
O    0.280
P    0.641
Name: Label, dtype: float64


In [123]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [130]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(800,)
(200,)
(800,)
(200,)


In [132]:
X_train.head

<bound method NDFrame.head of 29     So let me get this straight...Kindle is now a ...
535    i love my kindle.  i had to get a new purse to...
695    This is one of my favorite things about the ki...
557    Love it!! I can hold my.children and read!!! H...
836    Can someone please tell me if you can read you...
                             ...                        
106    Kindles rule , they should have them in school...
270    I read at least 3 books a week so the free boo...
860                                   Luv my Kindle !!!!
435    I bought one a few months ago and LOVE it! I f...
102                                     I love mine too.
Name: FBPost, Length: 800, dtype: object>

In [124]:
# Create a CountVectorizer object to convert text data to numerical data
# Perform text processing
# Create TF-IDF vectorizer

# Create feature vectors
vectorizer = TfidfVectorizer(stop_words='english', max_features=500)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Create feature vectors using the HashingVectorizer
#vectorizer = HashingVectorizer(n_features=2**20, alternate_sign=False, stop_words="english")
#X_train_vec = vectorizer.fit_transform(X_train)
#X_test_vec = vectorizer.transform(X_test)

In [125]:
# Perform oversampling with SMOTE
#oversampler = SMOTE(random_state=42)
#X_train, y_train = oversampler.fit_resample(X_train, y_train)

In [126]:
# Train an SVM model on the vectorized data
svm = SVC(kernel='linear', C=1.0)
svm.fit(X_train_vec, y_train)

SVC(kernel='linear')

In [127]:
# Train CatBoost model
#!pip install catboost
#from catboost import CatBoostClassifier

#model = CatBoostClassifier(random_state=42, task_type='GPU', iterations=1000, learning_rate=0.05, depth=6, loss_function='MultiClass')
#model.fit(X_train_vec, y_train, verbose=False)

In [128]:
# Make predictions on test data
X_test_vec = vectorizer.transform(X_test)
svm_pred = svm.predict(X_test_vec)

In [129]:
# Evaluate the accuracy, precision, and recall of the SVM model
svm_acc = accuracy_score(y_test, svm_pred)
svm_cm = confusion_matrix(y_test, svm_pred, labels=['O', 'P', 'N'])
svm_precisions = precision_score(y_test, svm_pred, average=None, labels=['O', 'P', 'N'])
svm_recalls = recall_score(y_test, svm_pred, average=None, labels=['O', 'P', 'N'])
f1_scores = f1_score(y_test, svm_pred, average=None)

# Print the accuracy, precision, and recall of the SVM model
print("SVM Accuracy:", svm_acc)
print("SVM Precision:", svm_precisions)
print("SVM Recall:", svm_recalls)
print("SVM Confusion Matrix:")
labels = ['O', 'P', 'N']
pd.DataFrame(svm_cm, index=labels, columns=labels)
print("F1 scores:")
for i, score in enumerate(f1_scores):
    print(f"Class {i}: {score:.3f}")

SVM Accuracy: 0.725
SVM Precision: [0.59375    0.78676471 0.        ]
SVM Recall: [0.6031746 0.856     0.       ]
SVM Confusion Matrix:
F1 scores:
Class 0: 0.000
Class 1: 0.598
Class 2: 0.820


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
